# Version information

In [ ]:
from datetime import date
print("Running date:", date.today().strftime("%B %d, %Y"))
import pyleecan
print("Pyleecan version:" + pyleecan.__version__)
import SciDataTool
print("SciDataTool version:" + SciDataTool.__version__)

# How to use MeshSolution objects

This tutorial shows the different possibilities allowed by the MeshSolution module. The main purpose of the module is to store Finite Element (FE) results in a way adapted to Pyleecan architecture. Today, it is mainly dedicated to store 2D electromagnetic solution computed with FEMM, but the goal is to generalize to any physics that could be included inside Pyleecan.
 
The notebook related to this tutorial is available on [GitHub](https://github.com/Eomys/pyleecan/tree/master/Tutorials/tuto_MeshSolution.ipynb).

This tutorial is for people who wish to understand in depth how this module works, and potentially contribute to the development of the code.

## Defining a Mesh object and plots

At the date of this webinar, there are two different types of Mesh objects: MeshMat and MeshVTK. 
- MeshMat object is designed to ease postprocessing. It enables access to important values (connectivity, nodes) and to defined interpolation methods. (numpy.array)
- MeshVTK is designed to ease vizualization, by relying on existing librairy pyvista.

The bridge between the two type of class is available with dedicated convert methods. 

### Defining a MeshMat object

Although every features should be automatically initialized/defined in Pyleecan, we are going to define by hand some of the objects in order to introduce the basics principle of the MeshSolution module.

In [ ]:
from pyleecan.Classes.MeshMat import MeshMat
from pyleecan.Classes.NodeMat import NodeMat
from pyleecan.Classes.CellMat import CellMat
from pyleecan.Classes.MeshSolution import MeshSolution

mesh = MeshMat(dimension=3)
mesh.node = NodeMat()
mesh.node.add_node([0, 0, 0])
mesh.node.add_node([0, 1, 0])
mesh.node.add_node([1, 0, 0])
mesh.node.add_node([1, 1, 0])
mesh.node.add_node([2, 1, 0])

mesh.cell["triangle"] = CellMat(nb_node_per_cell=3)
mesh.add_cell([0, 1, 2], "triangle")
mesh.add_cell([1, 2, 3], "triangle")
mesh.add_cell([2, 3, 4], "triangle")

MSol = MeshSolution(mesh=[mesh])

MSol.plot_mesh()

## Defining a SolutionMat object and plot
The MeshSolution object allows to make the link between data (such as FE results) and the corresponding mesh stored in a Mesh object. Thus, all the plot and post-processing methods should be available in the MeshSolution class.

Today, the main post-processing are the plots (such as plot_contour and plot_glyph).

Here is an example with plot_contour: a scalar field is defined by giving its values all points of the mesh. 

In [ ]:
import numpy as np
from pyleecan.Classes.SolutionMat import SolutionMat

axis_dct = dict()
axis_dct["indice"] = 5
axis_dct["time"] = 1
field = np.array([[0,1,2,3,4]])

my_solution = SolutionMat(
    label="my_field",
    type_cell="point",
    field=field,
    indice=[0, 1, 2, 3, 4],
    axis=axis_dct,
)
MSol.solution.append(my_solution)
MSol.plot_contour()

The notion of axis allows to correctly extract values as it would be with SciDataTool objects -> same way to call methods in SolutionMat/SolutionData/SolutionVector. 

Using SolutionMat, one can also defined a vector field by using an additional axis "component".

In [ ]:
new_axis_dct = dict()

new_axis_dct["time"] = 10
new_axis_dct["indice"] = 5
new_axis_dct["component"] = 2
vector = np.ones((10,5,2))

my_vec_solution = SolutionMat(
    label="my_vector",
    type_cell="point",
    field=vector,
    indice=[0, 1, 2, 3, 4], # optional today, but field size must match with the number of point/cell.
    axis=new_axis_dct,
)
MSol.solution.append(my_vec_solution)
MSol.plot_glyph(label="my_vector", is_point_arrow=True, factor=1/10)

In this example, a 2D field is defined on a 3D mesh. The mesh and the field has distinct "dimension" attributes. It allows us to limit the memory space when possible. 

In order to have more details about the intialization of SolutionData/SolutionVector objects, see SciDataTool part and build_meshsolution() method from MagFEMM. 

## Import an external Mesh

Today, Pyleecan mainly relies on the meshio librairy to convert any type of mesh file into a .vtk which is readable by pyvista. Any contribution on this topic is welcome.

In [ ]:
#Convert to vtk with meshio
import meshio
m = meshio.Mesh.read('my_mesh.msh')
m.write('my_mesh.vtk')

# Import in Pyleecan with MeshVTK
from pyleecan.Classes.MeshVTK import MeshVTK
mesh_import = MeshVTK(
    path="C:\\Users\\Raphael\\Desktop\\Git\\pyleecan_tests\\Tutorials\\",
    name="my_mesh",
)
MSol_import = MeshSolution(mesh=[mesh_import])
MSol_import.plot_mesh()

# Demo with FEMM results
The aim of this section is to show how MeshSolution object are used in Pyleecan to post-process FE results. 

In [ ]:
# Run the FEMM simulation from Webinar 1
from numpy import ones, pi, array, linspace, cos, sqrt
from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.MagFEMM import MagFEMM
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR

# Create the Simulation
IPMSM_A = load(join(DATA_DIR, "Machine", "IPMSM_A.json"))
simu_femm = Simu1(name="Webinar_1_MagFemm", machine=IPMSM_A)   
p = simu_femm.machine.stator.winding.p
qs = simu_femm.machine.stator.winding.qs

# Defining Simulation Input
simu_femm.input = InputCurrent()

# Rotor speed [rpm]
simu_femm.input.N0 = 2000

# time discretization [s]
time = linspace(start=0, stop=60/simu_femm.input.N0, num=32*p, endpoint=False) # 16 timesteps
simu_femm.input.time = time 

# Angular discretization along the airgap circonference for flux density calculation
simu_femm.input.angle = linspace(start = 0, stop = 2*pi, num=2048, endpoint=False) # 2048 steps 

# Stator currents as a function of time, each column correspond to one phase [A]
A_rms = 200 
felec = p * simu_femm.input.N0 /60 # [Hz]
rot_dir = simu_femm.machine.stator.comp_rot_dir()
Phi0 = 140*pi/180  # Maximum Torque Per Amp

Ia = (
    A_rms
    * sqrt(2)
    * cos(2 * pi * felec * time + 0 * rot_dir * 2 * pi / qs + Phi0)
)
Ib = (
    A_rms
    * sqrt(2)
    * cos(2 * pi * felec * time + 1 * rot_dir * 2 * pi / qs + Phi0)
)
Ic = (
    A_rms
    * sqrt(2)
    * cos(2 * pi * felec * time + 2 * rot_dir * 2 * pi / qs + Phi0)
)
simu_femm.input.Is = array([Ia, Ib, Ic]).transpose()


To enable the FE results saving: is_get_mesh 

In [ ]:
from pyleecan.Classes.MagFEMM import MagFEMM

simu_femm.mag = MagFEMM(
    is_get_mesh=True, # TO save FEMM mesh and results into a MeshSolution object
    type_BH_stator=0, 
    type_BH_rotor=0,
    is_periodicity_a=True,
    is_periodicity_t=True,
)

out_femm = simu_femm.run()

Now, the magnetic FEA results can be plotted. Moreover, the solution can be extracted on a specific area.

In [ ]:
out_femm.mag.meshsolution.plot_contour()

In [ ]:
out_femm.mag.meshsolution.plot_contour(label="H", group_names="stator core")

In [ ]:
out_femm.mag.meshsolution.plot_glyph(label="H", group_names="stator winding")

In [ ]:
out_femm.mag.meshsolution.plot_contour(label="B", group_names="airgap")

In [ ]:
out_femm.mag.meshsolution.plot_mesh(group_names=["stator core", "/", "airgap", "stator winding"])

# Extract and post-process data

Several methods have been developed for the MeshSolution class in order to load the results regardless of the type of objects.  

In [ ]:
B = out_femm.mag.meshsolution.get_field(label='B')
H = out_femm.mag.meshsolution.get_field(label='H')
B.shape

A new MeshSolution object can be created from the group definition. 

In [ ]:
group_stator = out_femm.mag.meshsolution.get_group("stator")
B_s = group_stator.get_field(label='B')
H_s = group_stator.get_field(label='H')
B_s.shape

Then, operations can be performed on the solution of this group, and plotted. It is worth noting that several type of Solution objects can co-exist in the same MeshSolution object.

In [ ]:
w_mag = np.multiply(B_s,H_s)/2

w_axis = dict()
w_axis["time"] = 16
w_axis["indice"] = 3801
w_axis["component"] = 2

my_vec_solution = SolutionMat(
    label="w_mag",
    type_cell="triangle",
    field=w_mag,
    axis=w_axis,
)
group_stator.solution.append(my_vec_solution)
group_stator.plot_contour(label="w_mag")

Operations can also be performed on the mesh.

In [ ]:
nodes_s = group_stator.get_mesh().get_point()
nodes_s.shape

Then, rotate the mesh

In [ ]:
th = np.pi
R = np.array([[np.cos(th), -np.sin(th)], [np.sin(th), np.cos(th)]])
nodes_s = np.dot(nodes_s, R)
group_stator.mesh[0].point.coordinate = nodes_s
group_stator.plot_mesh()

Previous plot still work !

In [ ]:
group_stator.plot_contour(label="w_mag")

Thanks for following this tutorial ! :-)